In [26]:
import tensorflow_data_validation as tfdv
import pandas as pd

In [35]:

# Paths
CSV_FILE_PATH = '/home/dino/Desktop/SP24/scripts/biden_stance_public_3_labeled.csv'
EVAL_CSV_FILE_PATH = '/home/dino/Desktop/SP24/scripts/biden_stance_test_public.csv'

In [36]:
# Load the training and evaluation datasets
train_df = pd.read_csv(CSV_FILE_PATH)
eval_df = pd.read_csv(EVAL_CSV_FILE_PATH)

In [37]:
train_stats = tfdv.generate_statistics_from_dataframe(train_df)
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

In [39]:
# Visualize the statistics of the training dataset
print("Training Dataset Statistics:")
tfdv.visualize_statistics(train_stats)

Training Dataset Statistics:


In [40]:
# Visualize the statistics of the evaluation dataset
print("Evaluation Dataset Statistics:")
tfdv.visualize_statistics(eval_stats)

Evaluation Dataset Statistics:


In [41]:
schema = tfdv.infer_schema(train_stats)

In [42]:
print("Inferred Schema:")
tfdv.display_schema(schema)

Inferred Schema:


,Type,Presence,Valency,Domain
Feature name,,,,
'tweet_id',INT,required,,-
'text',BYTES,required,,-
'label',STRING,required,,'label'


,Values
Domain,
'label',"'AGAINST', 'FAVOR', 'NEUTRAL'"


In [43]:
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)

In [44]:
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'label',Unexpected string values,Examples contain values missing from the schema: NONE (~41%).


In [45]:
eval_df['label'] = eval_df['label'].replace('NONE', 'NEUTRAL')

In [46]:
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'label',Unexpected string values,Examples contain values missing from the schema: NONE (~41%).


In [49]:
train_stats = tfdv.generate_statistics_from_dataframe(train_df)

In [50]:
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

In [51]:
schema = tfdv.infer_schema(train_stats)

In [52]:
train_anomalies = tfdv.validate_statistics(statistics=train_stats, schema=schema)

In [53]:
eval_anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)

In [54]:
# Display any anomalies found in the training dataset
print("Anomalies in Training Dataset:")
tfdv.display_anomalies(train_anomalies)

Anomalies in Training Dataset:


In [55]:
# Display any anomalies found in the evaluation dataset
print("Anomalies in Evaluation Dataset:")
tfdv.display_anomalies(eval_anomalies)

Anomalies in Evaluation Dataset:


In [58]:
# If you want to overwrite the original file, you can use:
eval_df.to_csv(EVAL_CSV_FILE_PATH, index=False)

In [59]:

# Define a path to save the schema
SCHEMA_FILE = '/home/dino/Desktop/SP24/scripts/schema.txt'

# Write the schema to a file
tfdv.write_schema_text(schema, SCHEMA_FILE)

In [60]:
import pkg_resources
import importlib
importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/lib/python3/dist-packages/pkg_resources/__init__.py'>

In [61]:
import os
import tempfile

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

from tfx_bsl.public import tfxio

In [62]:
import pandas as pd
import re


In [63]:
# Function to clean the text data
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\S+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove the '#' from hashtags
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces, tabs, and newlines
    return text

In [64]:
# Clean the 'text' column
df['text'] = df['text'].apply(clean_text)


In [65]:
# Overwrite the original file with the cleaned data
df.to_csv(CSV_FILE_PATH, index=False)